In [1]:
# packages
import sys
import os
import math
import torch
import random
import pickle
import calendar
import numpy as np
import config
import training 
import matplotlib.pyplot as plt
import evaluation as E
from torch.utils.data import TensorDataset, DataLoader
os.environ['KMP_DUPLICATE_LIB_OK']='True'
torch.set_default_tensor_type(torch.FloatTensor)

In [2]:
datasets = os.listdir('./dataset')
datasets = [f for f in datasets if (f.startswith('Dataset') and f.endswith('.p'))]
datasets.sort()
datasets

['Dataset_acuteinflammation.p',
 'Dataset_balancescale.p',
 'Dataset_breastcancerwisc.p',
 'Dataset_cardiotocography3clases.p',
 'Dataset_energyy1.p',
 'Dataset_energyy2.p',
 'Dataset_iris.p',
 'Dataset_mammographic.p',
 'Dataset_pendigits.p',
 'Dataset_seeds.p',
 'Dataset_tictactoe.p',
 'Dataset_vertebralcolumn2clases.p',
 'Dataset_vertebralcolumn3clases.p']

In [3]:
learning = 2

In [4]:
seeds = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
lr_rts = [0.005, 0.]
train_epsilons = [0., 0.05, 0.1]
test_epsilons = [0.05, 0.1]
strategies = [0]
Es = [0]

# Evaluation

In [5]:
results = torch.zeros([13, 10, 2, 3, 2, 1, 1, 2])

In [6]:
for i, dataset in enumerate(datasets):
    print(dataset)
    for j, seed in enumerate(seeds):
        for k, lr_rt in enumerate(lr_rts):
            for l, train_epsilon in enumerate(train_epsilons):
                for m, test_epsilon in enumerate(test_epsilons):
                    for n, strategy in enumerate(strategies):
                        datapath = os.path.join(f'./dataset/{dataset}')
                        with open(datapath, 'rb') as f:
                            data = pickle.load(f)
                        X_train    = data['X_train']
                        y_train    = data['y_train']
                        X_valid    = data['X_valid']
                        y_valid    = data['y_valid']
                        X_test     = data['X_test']
                        y_test     = data['y_test']
                        data_name  = data['name']

                        setup = f'dataset:{data_name}_epsilon:{train_epsilon}_strategy:{strategy}_learning:{learning}_lr_rt:{lr_rt}_seed:{seed}'

                        model = torch.load(f'./results_variation_circuit/pNN_{setup}')
                        model.SetParameter('N', config.N_test)
                        model.SetParameter('epsilon', test_epsilon)
                        
                        for o, e in enumerate(Es):
                            if o==0:
                                acc, std = E.BASIC_variation(model, X_test, y_test)
                            elif o==1:
                                acc, std = E.MAA_variation(model, X_test, y_test)
                            results[i,j,k,l,m,n,o,0], results[i,j,k,l,m,n,o,1] = acc, std

Dataset_acuteinflammation.p
Dataset_balancescale.p
Dataset_breastcancerwisc.p
Dataset_cardiotocography3clases.p
Dataset_energyy1.p
Dataset_energyy2.p
Dataset_iris.p
Dataset_mammographic.p
Dataset_pendigits.p
Dataset_seeds.p
Dataset_tictactoe.p
Dataset_vertebralcolumn2clases.p
Dataset_vertebralcolumn3clases.p


In [7]:
torch.save(results, f'./results_for_paper/EvaluationMatrix_{learning}')

In [8]:
results = torch.load(f'./results_for_paper/EvaluationMatrix_{learning}')
results.shape

torch.Size([13, 10, 2, 3, 2, 1, 1, 2])

select strategy and metric

In [9]:
results = results[:,:,:,:,:,0,0,:]
results.shape

torch.Size([13, 10, 2, 3, 2, 2])

In [10]:
results_learnable = results[:,:,0,:,:,:]
results_learnable.shape

torch.Size([13, 10, 3, 2, 2])

In [11]:
results_nonlearnable = results[:,:,1,:,:,:]
results_nonlearnable.shape

torch.Size([13, 10, 3, 2, 2])

# record results

In [12]:
final_results = np.zeros([13,2,4,2])

In [13]:
def find_best(results):
    best_seed = torch.argmax(results[:,:,0], dim=1)
    temp = []
    for i, j in zip(range(13),best_seed):
        temp.append(results[i,j,:].numpy())
    return np.array(temp)

## variation + learnable

5% variation

In [14]:
final_results[:,:,0,0] = find_best(results_learnable[:, :, 1, 0, :])

10% variation

In [15]:
final_results[:,:,0,1] = find_best(results_learnable[:, :, 2, 1, :])

## nominal + learnable

5% variation

In [16]:
final_results[:,:,1,0] = find_best(results_learnable[:, :, 0, 0, :])

10% variation

In [17]:
final_results[:,:,1,1] = find_best(results_learnable[:, :, 0, 1, :])

## variation + non-learnable

5% variation

In [18]:
final_results[:,:,2,0] = find_best(results_nonlearnable[:, :, 1, 0, :])

10% variation

In [19]:
final_results[:,:,2,1] = find_best(results_nonlearnable[:, :, 2, 1, :])

## non-variation + non-learnable

5% variation

In [20]:
final_results[:,:,3,0] = find_best(results_nonlearnable[:, :, 0, 0, :])

10% variation

In [21]:
final_results[:,:,3,1] = find_best(results_nonlearnable[:, :, 0, 1, :])

# final_results = [dataset, performance, experiment, variance]
13 datasets

2 performance: mean, std 

3 experiments: both, only lnc, only variation aware, neither

2 variances: 5% 10%

## Compare ablation study

In [22]:
np.savetxt(f'./results_for_paper/mean_5%_ACC_{learning}.txt', final_results[:,0,:,0], fmt='%.5f', delimiter='\t')
np.savetxt(f'./results_for_paper/std_5%_ACC_{learning}.txt', final_results[:,1,:,0], fmt='%.5f', delimiter='\t')
np.savetxt(f'./results_for_paper/mean_10%_ACC_{learning}.txt', final_results[:,0,:,1], fmt='%.5f', delimiter='\t')
np.savetxt(f'./results_for_paper/std_10%_ACC_{learning}.txt', final_results[:,1,:,1], fmt='%.5f', delimiter='\t')

In [25]:
final_results[:,0,:,0]

array([[1.        , 0.98199999, 0.92000002, 0.80320001],
       [0.88039684, 0.82944447, 0.84880954, 0.83515871],
       [0.96107143, 0.96421427, 0.93057144, 0.92957145],
       [0.76819247, 0.79983568, 0.7725352 , 0.77654928],
       [0.87792206, 0.89597404, 0.83766234, 0.81831169],
       [0.87279218, 0.86305195, 0.75415581, 0.72727275],
       [0.93064517, 0.84612906, 0.8429032 , 0.59225804],
       [0.77580309, 0.64917099, 0.74238342, 0.62284976],
       [0.59996361, 0.49457482, 0.47691223, 0.27465665],
       [0.76720929, 0.76279068, 0.57976747, 0.37186047],
       [0.63020831, 0.73677081, 0.63109374, 0.63020831],
       [0.6630159 , 0.72428572, 0.68555558, 0.68333334],
       [0.71111113, 0.55428571, 0.5177778 , 0.55746031]])

In [26]:
final_results[:,0,:,1]

array([[1.        , 0.91960001, 0.87919998, 0.79680002],
       [0.88111109, 0.77658731, 0.81753969, 0.70039684],
       [0.95264286, 0.92607141, 0.91628569, 0.89042854],
       [0.76291078, 0.71312207, 0.77046949, 0.77171361],
       [0.85402596, 0.87389612, 0.83655846, 0.73285717],
       [0.86532468, 0.79292208, 0.72766232, 0.72785717],
       [0.9029032 , 0.77387094, 0.78161293, 0.51290321],
       [0.7812435 , 0.59119171, 0.69673574, 0.58740932],
       [0.56505686, 0.37902227, 0.3227649 , 0.19977263],
       [0.83744186, 0.71906978, 0.75837207, 0.34627905],
       [0.63026041, 0.63109374, 0.63401043, 0.63020831],
       [0.63492066, 0.69555557, 0.67888892, 0.62714285],
       [0.64492065, 0.50174606, 0.58190477, 0.50793654]])